In [13]:

import pandas as pd
import numpy as np

df = pd.read_csv('Training Dataset\\Adani Enterprises.csv')
df = df.replace({',': ''}, regex=True)
df['Close'] = df['Close'].astype(float)
df['Open'] = df['Open'].astype(float)
df['High'] = df['High'].astype(float)
df['Low'] = df['Low'].astype(float)

short_window = 50
long_window = 200
target_pct = 0.02
stoploss_pct = 0.005

df['Short_EMA'] = df['Close'].ewm(span=short_window, adjust=False).mean()
df['Long_EMA'] = df['Close'].ewm(span=long_window, adjust=False).mean()

initial_cash = 1000
cash = initial_cash
position = 0
buy_price = 0
target_price = 0
stoploss_price = 0
total_value = []
accuracy_count = 0
total_trades = 0
trade_log = []

for i in range(max(short_window, long_window), len(df)):
    if df['Short_EMA'].iloc[i] > df['Long_EMA'].iloc[i]:
        if position == 0:
            if cash >= df['Close'].iloc[i]:
                position = 1
                cash -= df['Close'].iloc[i]
                buy_price = df['Close'].iloc[i]
                target_price = buy_price * (1 + target_pct)
                stoploss_price = buy_price * (1 - stoploss_pct)
                total_trades += 1
                trade_log.append({
                    'Date': df.index[i],
                    'Action': 'Buy',
                    'Price': buy_price,
                    'Target_Price': target_price,
                    'Stoploss_Price': stoploss_price,
                    'Trade_Result': 'Bought'
                })

    if position > 0:
        trade_result = ''
        if df['Close'].iloc[i] >= target_price:
            cash += position * df['Close'].iloc[i]
            position = 0
            trade_result = 'Target Hit'
            accuracy_count += 1
        elif df['Close'].iloc[i] <= stoploss_price:
            cash += position * df['Close'].iloc[i]
            position = 0
            trade_result = 'Stoploss Hit'
        elif df['Short_EMA'].iloc[i] < df['Long_EMA'].iloc[i]:
            cash += position * df['Close'].iloc[i]
            position = 0
            trade_result = 'Crossover'

        if trade_result != '':
            trade_log.append({
                'Date': df.index[i],
                'Action': 'Sell',
                'Price': df['Close'].iloc[i],
                'Target_Price': target_price,
                'Stoploss_Price': stoploss_price,
                'Trade_Result': trade_result
            })

    total_value.append(cash + position * df['Close'].iloc[i])

if total_trades == 0:
    print("No trades executed due to insufficient cash.")
else:
    final_value = total_value[-1]
    total_return = (final_value - initial_cash) / initial_cash * 100
    accuracy = accuracy_count / total_trades * 100

    print(f"Initial Cash: ₹{initial_cash}")
    print(f"Final Portfolio Value: ₹{final_value:.2f}")
    print(f"Total Return: {total_return:.2f}%")
    print(f"Total Trades: {total_trades}")
    print(f"Winning Trades: {accuracy_count}")
    print(f"Accuracy: {accuracy:.2f}%")

    import numpy as np
    portfolio_values = np.array(total_value)
    daily_returns = np.diff(portfolio_values) / portfolio_values[:-1]
    risk_free_rate_annual = 0.05
    trading_days = 252
    risk_free_rate_daily = risk_free_rate_annual / trading_days
    excess_daily_returns = daily_returns - risk_free_rate_daily
    if np.std(daily_returns) != 0:
        sharpe_ratio = np.mean(excess_daily_returns) / np.std(daily_returns) * np.sqrt(trading_days)
    else:
        sharpe_ratio = 0
    print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

trade_df = pd.DataFrame(trade_log)
trade_df.to_csv('ema_trade_log_with_results_3.csv', index=False)
print("Trade log with results has been saved to 'ema_trade_log_with_results_3.csv'.")

Initial Cash: ₹1000
Final Portfolio Value: ₹2139.45
Total Return: 113.94%
Total Trades: 252
Winning Trades: 104
Accuracy: 41.27%
Sharpe Ratio: 1.34
Trade log with results has been saved to 'ema_trade_log_with_results_3.csv'.
